# Aula 8: Estruturas de Dados Avançadas

Na Unidade 3, abordamos estruturas que suportam algoritmos de alta performance em cenários de grande volume de dados.

## Objetivos de Aprendizagem

Ao final desta aula, você deverá ser capaz de:
1. Entender a importância de estruturas balanceadas para operações em tempo logarítmico.
2. Definir fatores de balanceamento e implementar rotações em árvores AVL.
3. Inserir elementos em AVL com rebalanceamento automático.
4. Compreender e usar Heaps (min-heap / max-heap) para filas de prioridade.
5. Ter uma visão geral de B-Trees e sua aplicação em bancos de dados.

## 1. Parte 1/3: Árvores AVL – Revisão de Rotações

### Conceitos de AVL e Tipos de Rotações
- Fator de Balanceamento (FB) = h(esq) – h(dir), deve estar em [-1,0,1].
- Casos: LL, RR, LR, RL com rotações simples ou duplas.

In [ ]:
class AVLNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.height = 1  # nó inicial como folha

def height(node):
    return node.height if node else 0

def update_height(node):
    node.height = 1 + max(height(node.left), height(node.right))

def rotate_right(y):
    x = y.left
    T2 = x.right
    x.right = y
    y.left = T2
    update_height(y)
    update_height(x)
    return x

def rotate_left(x):
    y = x.right
    T2 = y.left
    y.left = x
    x.right = T2
    update_height(x)
    update_height(y)
    return y


#### Parte 1 de 3 completa. A seguir, implementamos inserção com rebalanceamento.

## 2. Parte 2/3: Inserção em AVL com Rebalanceamento

Ao inserir um novo nó, atualizamos alturas e checamos FB de cada ancestro, aplicando rotações se necessário.

In [ ]:
def insert_avl(node, key):
    if not node: return AVLNode(key)
    if key < node.key:
        node.left = insert_avl(node.left, key)
    else:
        node.right = insert_avl(node.right, key)
    update_height(node)
    fb = height(node.left) - height(node.right)
    if fb > 1 and key < node.left.key:
        return rotate_right(node)  # LL
    if fb < -1 and key > node.right.key:
        return rotate_left(node)   # RR
    if fb > 1 and key > node.left.key:
        node.left = rotate_left(node.left)  # LR
        return rotate_right(node)
    if fb < -1 and key < node.right.key:
        node.right = rotate_right(node.right)  # RL
        return rotate_left(node)
    return node

# Teste de inserção
root = None
for v in [10,20,30,40,50,25]:
    root = insert_avl(root, v)
print('AVL inserido com rebalanceamento concluído')

**Análise**: Cada inserção custa $O(\log n)$.

#### Parte 2 de 3 completa. Na Parte 3, veremos Heaps e B-Trees.

## 3. Parte 3/3: Heaps e B-Trees

### 3.1 Heaps (Min-Heap e Max-Heap)
- **Min-Heap**: raiz contém o menor elemento.
- **Max-Heap**: raiz contém o maior elemento (implementado com valores negativos em min-heap).
- Operações: `push`, `pop` em $O(\log n)$; acesso ao topo em $O(1)$.

In [ ]:
import heapq
# Min-Heap
nums = [5, 1, 7, 3, 9]
min_heap = []
for x in nums:
    heapq.heappush(min_heap, x)
print('Min-Heap topo:', min_heap[0])
# Pop elemento mínimo
print('Pop min:', heapq.heappop(min_heap))

# Max-Heap via min-heap com valores negativos
max_heap = []
for x in nums:
    heapq.heappush(max_heap, -x)
print('Max-Heap topo:', -max_heap[0])
print('Pop max:', -heapq.heappop(max_heap))

### 3.2 B-Trees – Visão Geral
- **Árvore multiway**: cada nó pode ter até `m` filhos, otimizada para acesso em disco.
- **Propriedades**: todos os nós folhas no mesmo nível; número de chaves em cada nó entre `⌈m/2⌉-1` e `m-1`.
- **Uso**: índices de bancos de dados e sistemas de arquivos, minimizando leituras de disco.

**Pseudocódigo**: Inserção em B-Tree  order `m`:
```pseudocode
B_TREE_INSERT(T, k):
  r = T.root
  if r.n == 2t-1:
    s = ALLOC_NODE()
    T.root = s
    s.leaf = False
    s.n = 0
    s.child[0] = r
    SPLIT_CHILD(s, 0)
    INSERT_NONFULL(s, k)
  else:
    INSERT_NONFULL(r, k)
```